In [61]:
import pandas as pd
import numpy as np

data_path = '../data/'

sales_train = pd.read_csv(data_path+'sales_train.csv')
shops = pd.read_csv(data_path+'shops.csv')
items = pd.read_csv(data_path+'items.csv')
item_categories = pd.read_csv(data_path+'item_categories.csv')
test = pd.read_csv(data_path+'test.csv')
submission = pd.read_csv(data_path+'sample_submission.csv')


In [62]:
sales_train = sales_train.rename(columns = {'date': '날짜',
                                            'date_block_num': '월ID',
                                            'shop_id': '상점ID',
                                            'item_id': '상품ID',
                                            'item_price': '판매가',
                                            'item_cnt_day': '판매량'})
shops = shops.rename(columns = {'shop_name':'상점명',
                                'shop_id': '상점 ID'},)
items = items.rename(columns={
    'item_name':'상품명',
    'item_id': '상점ID',
    'item_category_id':'상품분류ID'
})
item_categories = item_categories.rename(columns = {'item_category_name':'상품 분류명', 'item_category_id': '상품 분류 ID'})
test = test.rename(columns ={'shop_id':'상점 ID','item_id':'상품 ID'})

In [63]:
def downcast(df, verbose = True):
    start_mem = df.memory_usage().sum()/1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast = 'integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast= 'float')
    end_mem = df.memory_usage().sum()/1024**2
    if verbose:
        print('{:.1f}% 압축됨'.format(100*(start_mem-end_mem)/start_mem))
    return df

In [ ]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df=downcast(df)

In [ ]:
from itertools import product

train = []

for i in sales_train['월ID'].unique():
    all_shop = sales_train.loc[sales_train['월ID']==i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID']==i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

idx_features = ['월ID', '상점ID', '상품ID']

train = pd.DataFrame(np.vstack(train),columns = idx_features)

train
